# Frequency input
Put together the letter and bigram frequency files based on the data that comes from Mathieu. Distributes the frequency of combined characters that are not in the to-be-mapped character set of the scenario and filters out characters that are irrelevant.

In [ ]:
from read_input import *
import codecs

letter_file = "input/resources/frequency_letters_mathieu.txt"
bigram_file = "input/resources/frequency_bigrams_mathieu.txt"
letters = get_letters()

scenario="scenario1"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario)

scenario="scenario2"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario)

scenario="scenario3"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario)

No frequency for θ2
5000 of 304793
10000 of 304793